In [ ]:
import seaborn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import Counter

In [ ]:
def load_file(path) -> pd.DataFrame:
    df = pd.read_excel(path, header=0)
    df['DateTime']= pd.to_datetime(df['Date'].astype(str)+' '+ df['Time'].astype(str))
    df = df.drop(['Date', 'Time'], axis=1)
    return df


def plot_timedelta_distr(df: pd.DataFrame):
    delta_arr = []
    for i in range(1, len(df)):
        delta_arr.append(df.iloc[i, -1] - df.iloc[i-1, -1])
    print(Counter(delta_arr), '\n')
    hist_arr = [i.total_seconds()/60 for i in delta_arr]
    plt.hist(hist_arr)
    plt.show()


def construct_XY(df_X: pd.DataFrame, df_Y: pd.DataFrame, max_diff: pd.Timedelta, diff2: pd.Timedelta):
    num = 0
    Y = df_Y.iloc[num,:-1].to_numpy()
    X = np.array([],).reshape(0, len(df_X.columns)-1)
    for i in range(len(df_X)):
        timeX = df_X.iloc[i, -1]
        timeY = df_Y.iloc[num, -1]
        if timeX > timeY:
            if timeX - timeY < diff2:
                tmp_X = df_X.iloc[i, :-1].to_numpy()
                X = np.vstack((X, tmp_X))
            yield X, Y
            num += 1            
            if num >= len(df_Y):
                break
            
            X = np.array([],).reshape(0, len(df_X.columns)-1)
            Y = df_Y.iloc[num,:-1].to_numpy()
            continue
        if timeY - timeX > max_diff:
            continue
        tmp_X = df_X.iloc[i, :-1].to_numpy()
        X = np.vstack((X, tmp_X))


def plot_num_distr(gen):
    shapes = []
    i = 0
    for x, y in gen:
        i += 1
        shapes.append(len(x))
    print(f"Количество точек {i} \n")
    print(f'Распределение количества примеров \n {Counter(shapes)} \n')
    plt.hist(shapes)
    plt.plot()

In [ ]:
df_Y1 = load_file('Лабораторные анализы Давление насыщенных паров в продукте, зимний период.xlsx')
df_Y2 = load_file('Лабораторные анализы Конец кипения легкого бензина.xlsx')
df_Y3 = load_file('Лабораторные анализы Содержание олефинов в продукте.xlsx')

df_X1 = load_file('Процессные данные Давление насыщенных паров в продукте, зимний период.xlsx')
df_X2 = load_file('Процессные данные Конец кипения легкого бензина.xlsx')
df_X3 = load_file('Процессные данные Содержание олефинов в продукте.xlsx')

In [ ]:
print(df_Y1.isnull().any().any())
print(df_Y2.isnull().any().any())
print(df_Y3.isnull().any().any())

print(df_X1.isnull().any().any())
print(df_X2.isnull().any().any())
print(df_X3.isnull().any().any())

In [ ]:
df_Y1

In [ ]:
df_X1

In [ ]:
plot_timedelta_distr(df_X1)

In [ ]:
diff = pd.to_timedelta('3.5h')
diff2 = pd.to_timedelta('0.5h')
gen_1 = construct_XY(df_X1, df_Y1, diff, diff2)
gen_2 = construct_XY(df_X2, df_Y2, diff, diff2)
gen_3 = construct_XY(df_X3, df_Y3, diff, diff2)

In [ ]:
plot_num_distr(construct_XY(df_X1, df_Y1, diff, diff2))

In [ ]:
plot_num_distr(construct_XY(df_X2, df_Y2, diff, diff2))

In [ ]:
plot_num_distr(construct_XY(df_X3, df_Y3, diff, diff2))